#Import Library

In [ ]:
from tabulate import tabulate
import matplotlib.pyplot as plt, numpy as np
import sympy
from sympy import symbols, Eq
from sympy import diff, Symbol
from sympy.parsing.sympy_parser import parse_expr
from sympy.abc import x
from sympy.utilities.lambdify import implemented_function
from sympy import lambdify
from numpy import sin, cos, tan, log, exp, pi
from numpy import pi

#Define Variables

In [ ]:
formula = ""
backupFormula = ""
x1 = ""
df_string = ""

#Functions

In [ ]:
def f(x):
  return eval(formula)

def df(x):
  x1 = sympy.symbols("x")
  df_string = str(sympy.diff(formula, x1))
  return eval(df_string)

In [ ]:
def LagrangePolynomial(x, x_points, y_points):
  pol = 0
  n = len(x_points)
  for k in range(n):
    L = 1
    for i in range(n):
      if i!=k:
        L *= ((x-x_points[i])/(x_points[k]-x_points[i]))
    pol += y_points[k]*L
  return pol

In [ ]:
def Neville1(tabel_lama, list_x, x_target, x_baru, y_baru):
  list_x_baru = []
  for nilai in list_x:
    list_x_baru.append(nilai)

  list_x_baru.append(x_baru)

  tabel_baru = []
  for baris in tabel_lama:
    calon_baris = []
    for nilai in baris:
      calon_baris.append(nilai)
    tabel_baru.append(calon_baris)

  for i in range(len(tabel_baru)):
    tabel_baru[i].append("")

  baris_baru = [y_baru]

  for i in range(len(tabel_baru)):
    baris_baru.append("")

  tabel_baru.append(baris_baru)

  k = len(baris_baru)
  i = k-1
  for j in range(1, k):
    pembilang_kiri = (x_target - list_x_baru[i-j]) * tabel_baru[i][j-1]
    pembilang_kanan = (x_target - list_x_baru[i]) * tabel_baru[i-1][j-1]
    pembilang = pembilang_kiri - pembilang_kanan
    tabel_baru[i][j] = (pembilang)/(list_x_baru[i] - list_x_baru[i-j])

  return tabel_baru, list_x_baru

def Neville2(tabel_utama, list_x):
  k = len(tabel_utama)
  tabel_lengkap = []
  for baris in tabel_utama:
    calon_baris = []
    for nilai in baris:
      calon_baris.append(nilai)
    tabel_lengkap.append(calon_baris)

  for i in range(k):
    tabel_lengkap[i].insert(0, list_x[i])
    tabel_lengkap[i].insert(0, i)

  header_list = ["i", "x_i"]
  for j in range(k):
    header_list.append("Q_{i," + str(j) + "}")

  tabel_olahan = tabulate(
    tabel_lengkap,
    headers = header_list,
    tablefmt = "orgtbl"
  )
  return tabel_olahan

def MetodeNeville(list_x_awal, list_y, x_target):
  tabel_utama = [
    [list_y[0]]
  ]
  list_x = []
  for nilai in list_x_awal:
    list_x.append(nilai)

  k = len(list_x)

  for i in range(1, k):
    tabel_utama, temp = Neville1(
      tabel_utama,
      list_x[:i],
      x_target,
      list_x[i], list_y[i]
    )

  tabel_olahan = Neville2(tabel_utama, list_x)
  print("Tabel Metode Interpolasi Neville:")
  print(tabel_olahan)

In [ ]:
def BuatTabelHermitteDD(list_x_asli, list_y_asli, list_p):
  list_x = []

  list_y = []
  list_pz = []
  for i in range(len(list_x_asli)):
    list_x.append(list_x_asli[i])
    list_x.append(list_x_asli[i])
    list_y.append(list_y_asli[i])
    list_y.append(list_y_asli[i])

    list_pz.append(list_p[i])
    list_pz.append(list_p[i])

  TabelUtamaDD = []
  banyak_data = len(list_x)
  n = banyak_data - 1

  for i in range(2*n+1):
    calon_baris = []
    for j in range(n+1):
      calon_baris.append("")
    TabelUtamaDD.append(calon_baris)

  for i in range(0, 2*n+1, 2):
    TabelUtamaDD[i][0] = list_y[int(i/2)]

  for j in range(1, n+1):
    a = 0
    b = j
    for i in range(j, 2*n - j + 1, 2):
      if (((b-a)==1) and (a % 2 == 0)):
        TabelUtamaDD[i][j] = list_pz[a] # f'(z_a)
      else:
        TabelUtamaDD[i][j] = (TabelUtamaDD[i+1][j-1] - TabelUtamaDD[i-1][j-1])/(list_x[b] - list_x[a])

      a += 1
      b += 1

  TabelDD = []
  for row in TabelUtamaDD:
    calon_baris = []
    for col in row:
      calon_baris.append(col)
    TabelDD.append(calon_baris)

  for i in range(2*n+1):
    indeks_x = int(i/2)
    if i % 2 == 0:
      TabelDD[i].insert(0, list_x[indeks_x])
      TabelDD[i].insert(0, indeks_x)
    else:
      TabelDD[i].insert(0, "")
      TabelDD[i].insert(0, "")

  semua_header = ["i", "z_i", "f(z_i)"]
  for k in range(1, n+1):
    semua_header.append("DD ke-" + str(k))

  olahan_TabelDD = tabulate(TabelDD, headers=semua_header,
                               tablefmt="orgtbl")

  return TabelUtamaDD, olahan_TabelDD

In [ ]:
def BackwardDD(tabel_utama, list_x):
  x = sympy.symbols('x')
  banyak_data = len(tabel_utama[0])
  n = banyak_data - 1
  pol = 0
  for k in range(banyak_data):
    suku = tabel_utama[2*n-k][k]
    for j in range(n-k+1, n+1):
      suku *= (x - list_x[j])
    pol += suku
  return pol

In [ ]:
def InterpolasiLagrangeBiasa(a, b, titik, titikFx, aproksimasi):
  print('=== HASIL INTERPOLASI LAGRANGE BIASA ===\n')
  x = sympy.symbols('x')
  y_lagrange = LagrangePolynomial(x, titik, titikFx)
  y_sederhana = sympy.simplify(y_lagrange)
  y_function = sympy.lambdify(x, y_sederhana)
  nilai_y = y_function(aproksimasi)

  sympy.pprint(y_lagrange)
  print("Disederhanakan:")
  sympy.pprint(y_sederhana)

  formula = str(y_sederhana)
  getF = lambda x : eval(formula)
  x = np.linspace(a, b, 100)
  y = []
  for i in range(len(x)):
    y.append(getF(x[i]))
  plt.plot(x, y, 'b-')

  print()
  plt.show()

  print()
  print("Aproksimasi nilai fungsi di x = {0} adalah y = {1:.5f}".format(aproksimasi, nilai_y))

  return y_sederhana

In [ ]:
def Neville(titik, titikFx, aproksimasi):
  print("=== HASIL METODE INTERPOLASI NEVILLE ===")
  MetodeNeville(titik, titikFx, aproksimasi)

In [ ]:
def HermiteDividedDifferences(titik, titikFx, titikDFx):
  print("=== HASIL HERMITE DIVIDED DIFFERENCES ===")
  tabel_utama, tabel_olahan = BuatTabelHermitteDD(titik, titikFx, titikDFx)
  print(tabel_olahan)
  return tabel_utama

In [ ]:
def PolinomHermitteBackwardDD(a, b, tabel_utama, titik, aproksimasi):
  print('Polinom Hermitte Backward Divided Differences:\n')
  titik_x = []
  for i in range(len(titik)):
    titik_x.append(titik[i])
    titik_x.append(titik[i])

  x = sympy.symbols('x')
  back_pol = BackwardDD(tabel_utama, titik_x)

  back_sederhana = sympy.simplify(back_pol)
  back_function = sympy.lambdify(x, back_sederhana)
  nilai_back = back_function(aproksimasi)

  print("Polinom Hasil Backward Divided Differences:")
  sympy.pprint(back_pol)
  print()
  print("disederhanakan:")
  sympy.pprint(back_sederhana)
  print()

  print('Plot fungsi:\n')

  x = np.linspace(a, b, 100)
  y = []
  formula = str(back_sederhana)
  getF = lambda x : eval(formula)
  print(formula)
  for i in range(len(x)):
    y.append(getF(x[i]))
  plt.plot(x, y, 'b-')

  print()
  plt.show(

  )

  print()
  print("Nilai Hermitte Backward Divided Differences pada x = {0}:".format(aproksimasi))
  print(nilai_back)

  return back_sederhana

In [ ]:
def SemuaMetode(a, b, y_sederhana, back_sederhana, formula):
  print('=== PLOT SEMUA (kecuali Metode Interpolasi Neville) ===\n')
  getF = lambda x : eval(formula)

  backupFormula = formula
  formula = str(y_sederhana)
  x = np.linspace(a, b, 500)
  y = []
  for i in range(len(x)):
    y.append(getF(x[i]))
  plt.plot(x, y, 'b-')

  x = np.linspace(a, b, 500)
  y = []
  formula = str(back_sederhana)
  for i in range(len(x)):
    y.append(getF(x[i]))
  plt.plot(x, y, 'y-')

  x = np.linspace(a, b, 500)
  y = []
  formula = backupFormula
  for i in range(len(x)):
    y.append(getF(x[i]))
  plt.plot(x, y, 'g-')

  plt.legend(['Lagrange Biasa', 'Hermitte Backward Divided Differences', 'Fungsi Asli'])

  plt.show()

In [ ]:
def ubahT(str_f, t):
  result = ""
  expr = "+-*/%"
  for i in range(len(str_f)):
    if str_f[i] == 't':
      if str_f[i-1].isnumeric():
        result += '*' + t
      elif i < len(str_f)-1 and str_f[i+1].isnumeric():
        result += t + '*'
      elif str_f[i-1] == ' ' and i < len(str_f)-1 and (str_f[i+1] == ' ' or str_f[i] in expr):
        result += t
      elif str_f[i-1].isalpha() or (i < len(str_f)-1 and str_f[i+1].isalpha()):
        result += str_f[i]
      else:
        result += t
    else:
      result += str_f[i]
  return result

In [ ]:
while True:
  npm = input("Masukkan NPM Anda : ")
  t = int(npm) % 10

  print('Maka, t =', t)
  print('Masukkan sembarang fungsi f(x), boleh bergantung t:')
  formula = input('f(x) = ')

  formula = ubahT(formula, str(t))

  backupFormula = formula

  print('Maka, f(x) = {}\n'.format(formula))

  print('Fungsi f(x) akan diaproksimasi dengan polinom menggunakan metode interpolasi berikut:')
  print('- Metode Interpolasi Lagrange Biasa')
  print('- Metode Interpolasi Neville')
  print('- Metode Interpolasi Hermitte Backward Divided Difference')

  while True:
    n = int(input('Masukkan banyak titik yang ingin digunakan untuk interpolasi: '))

    if n > 1:
      break
    else:
      print('Banyak titik harus lebih dari atau sama dengan 2')

  print('Akan digunakan {} titik dari suatu interval [a, b].'.format(n))
  a = int(input('Masukkan (batas bawah) a: '))
  b = int(input('Masukkan (batas atas) b: '))
  titik = [a]
  nilaiBeda = ((b - a) / (n - 1))
  for i in range(1, n-1):
    titik.append(titik[i-1] + nilaiBeda)

  titik.append(b)

  print('Titik-titik yang akan digunakan untuk interpolasi adalah:')
  print('{}'.format(titik))

  print()
  print('Diperoleh nilai f(x) pada titik-titik tersebut:')
  titikFx = []
  for i in range(len(titik)):
    titikFx.append(f(titik[i]))

  print(titikFx)

  titikDFx = []
  for i in range(len(titik)):
    titikDFx.append(df(titik[i]))

  print("Nilai f'(x) pada titik-titik tersebut:")
  print('{}'.format(titikDFx))

  print()
  aproksimasi = eval(input('Masukkan nilai x yang ingin dihitung hasil aproksimasinya: '))
  print('Nilai f({}) akan diaproksimasi.'.format(aproksimasi))

  print()
  y_sederhana = InterpolasiLagrangeBiasa(a, b, titik, titikFx, aproksimasi)

  print()
  Neville(titik, titikFx, aproksimasi)

  print()
  tabel_utama = HermiteDividedDifferences(titik, titikFx, titikDFx)

  print()
  back_sederhana = PolinomHermitteBackwardDD(a, b, tabel_utama, titik, aproksimasi)

  print()
  SemuaMetode(a, b, y_sederhana, back_sederhana, backupFormula)

  print()
  while True:
    choice = input("Apakah Anda ingin mengulang program? (y/n) ")
    if choice == 'n' or choice == 'y':
      break

  if choice == 'n':
    break
  else:
    print()

print('Terima kasih telah menggunakan program.')